La finalidad de este segundo cuaderno es hacerle finetuning a un modelo basado en bert, entrenado en español (bert-base-spanish) para predecir la "dificultad" de un prompt que se le hace a nuestra herramienta "Midas Help". Así como si la pregunta no está relacionada con el TFM.

De esta forma, se optimizan costes al usar LLMs más caros (e inteligentes) unicamente cuando es necesario. Así como no llamar a ningun LLM y ofrecer una respuesta estática en caso de que se utilice para otros fines.

## 1. Importar lo necesario

Librerias necesarias etc.

In [1]:
# Celda 1: Importar dependencias
import pandas as pd
import numpy as np
import torch
import re
import copy
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
import unicodedata

## 2. Generar dataset
Lo definiremos aqui mismo. Los valores de "dificultad" serán:

0 -> Pregunta "fácil"

1 -> Pregunta "dificil"

2 -> Pregunta no relacionada.

Ejemplos de preguntas serían:
* Facil: "¿cual es el enlace al repositorio github de MIDAS?"
* Dificil: "¿cuántos agentes tiene definidos midas touch? ¿cómo funciona?"
* No relacionado: "dame una receta de lasaña".

In [2]:
# Celda 2: Generar dataset

# Prompts para dificultad 0 (preguntas fáciles)
# Preguntas fáciles sobre Midas (prompts_diff0)
prompts_diff0 = [
    # Preguntas básicas con diferentes formulaciones
    "¿Qué es Midas?",
    "qué es esto de MIDAS",
    "dime que es midas",
    "explicame brevemente midas",
    "midas es un sistema de ML?",
    "para q sirve midas",
    "MIDAS q hace exactamente",
    "q hace este sistema?",
    "me puedes decir qué es MIDAS?",
    "qué significa MIDAS como siglas?",
    "de qué trata MIDAS?",
    "quien ha hecho midas",
    "cuantas personas han trabajado en este proyecto",

    # Preguntas sobre TFM
    "¿Qué significa TFM en este contexto?",
    "a qué se refiere TFM",
    "esto es un TFM?",
    "TFM es trabajo fin de master?",
    "de que universidad es este TFM?",
    "q es un TFM?",

    # Conteo y componentes
    "¿Cuántos componentes tiene Midas?",
    "componentes de midas",
    "cuantas partes tiene este sistema",
    "cuantos modulos forman midas",
    "dime los componentes de midas",
    "lista las partes del sistema",
    "tiene 8 componentes?",
    "cuantos módulos tiene?",
    "me puedes contar cuántas partes tiene?",

    # Preguntas de seguimiento/conversacionales
    "y eso qué hace?",
    "explícame más",
    "no entiendo, dime más sobre eso",
    "puedes ampliar la explicación?",
    "me das un ejemplo?",
    "cómo funciona?",
    "y para qué sirve eso?",
    "qué más puede hacer?",
    "entiendo, y cómo lo implemento?",
    "y eso en qué me ayuda?",
    "ok y cómo lo uso?",
    "me explicas mejor?",
    "cómo así?",

    # Preguntas sobre componentes específicos
    "¿Para qué sirve Midas Help?",
    "midas help q hace",
    "q hace el componente help",
    "explícame midas help",
    "help hace lo mismo q tu?",
    "midas architect para q sirve",
    "¿Qué objetivo tiene Midas Architect?",
    "q es midas architect",
    "para q se usa el componente architect",
    "¿Qué hace el componente Midas Deploy?",
    "q hace midas deploy",
    "deploy de q se encarga",
    "Midas Touch q hace",
    "¿Qué hace Midas Test?",
    "test para q sirve",
    "midas dataset q genera?",
    "¿De qué manera Midas Dataset genera datasets?",
    "dataset cómo funciona",
    "¿Que componente genera las gráficas?",
    "cual es el q hace graficos",
    "q componente visualiza datos",
    "quien crea las visualizaciones",
    "¿Cómo ayuda Midas Assistant?",
    "asistente para q sirve",
    "q hace el assistant",

    # Preguntas sobre arquitectura y tecnología
    "¿En qué se diferencia Midas de otros sistemas ML?",
    "q tiene de especial midas",
    "por qué usar midas y no algo normal",
    "ventajas de midas vs otros sistemas",
    "¿Qué lenguaje usa Midas?",
    "esta hecho en q lenguaje",
    "lenguaje de programacion de midas",
    "usa python?",
    "¿Se puede usar con Python 3.8?",
    "funciona en python 3.8?",
    "versiones de python compatibles",
    "con q python lo ejecuto",

    # Preguntas sobre recursos y disponibilidad
    "¿Cuál es la página web de Midas?",
    "tiene web?",
    "donde puedo ver info online",
    "hay alguna web oficial?",
    "¿Dónde se almacenan los repositorios?",
    "donde está el repo",
    "github de midas",
    "git del proyecto",
    "link del repositorio",
    "url del proyecto",

    # Preguntas sobre instalación y uso
    "¿Los componentes se instalan por separado?",
    "como instalo midas",
    "se instala todo junto?",
    "instalacion por componentes?",
    "puedo instalar solo una parte?",
    "¿Cómo se integra con los datos?",
    "como uso mis datos con esto",
    "puedo usar mis propios datasets?",
    "¿Puedo usar datos en CSV?",
    "formatos de datos compatibles",
    "acepta excel?",
    "puedo usar json?",

    # Preguntas sobre requisitos y funcionamiento
    "¿Existen requisitos de hardware?",
    "q pc necesito para usarlo",
    "hardware recomendado",
    "me funcionará en mi laptop?",
    "RAM necesaria",
    "necesito una gpu?",
    "¿Se requieren librerías externas?",
    "dependencias necesarias",
    "q librerias hay q instalar",
    "paquetes de python q necesita",

    # Preguntas sobre uso específico
    "¿Cuáles son los pasos para entrenar un modelo?",
    "cómo entreno un modelo",
    "pasos para crear un modelo",
    "tutorial para entrenar modelo",
    "¿Cómo se guardan los modelos?",
    "donde guarda los modelos?",
    "formato de guardado",
    "extension del modelo",
    "usa pickle o joblib?",
    "¿Puedo crear un modelo de clasificación?",
    "sirve para clasificacion",
    "puedo hacer regresion?",
    "tipos de modelos q crea",

    # Preguntas sobre documentación
    "¿Hay un manual de usuario?",
    "donde esta la documentacion",
    "tutorial de uso",
    "guia de usuario",
    "¿Dónde encuentro la documentación?",
    "docs de midas",
    "tienen documentacion en español?",

    # Preguntas sobre características específicas
    "¿La capa de clasificación está en Midas Help?",
    "q es la capa de deteccion de dificultad",
    "para q sirve detectar dificultad",
    "¿Puedo crear múltiples joblibs?",
    "cuantos modelos puedo entrenar",
    "¿Los agentes se comunican entre sí?",
    "comunicacion entre agentes",
    "como interactuan los componentes",

    # Preguntas sobre capacidades y limitaciones
    "¿Qué tamaño de dataset es recomendable?",
    "dataset maximo q soporta",
    "cuantas filas aguanta",
    "¿Es difícil de configurar?",
    "configuracion complicada?",
    "es facil de usar?",
    "tiempo de configuracion",
    "¿Necesito conocimientos avanzados de ML?",
    "nivel de conocimiento necesario",
    "sirve para principiantes?",
    "es para expertos?",

    # Preguntas sobre exportación y visualización
    "¿Puedo exportar resultados a CSV?",
    "exportar a excel",
    "como saco los resultados",
    "formatos de exportacion",
    "¿Midas Plot muestra correlaciones?",
    "tipos de graficos disponibles",
    "visualizaciones posibles",
    "q graficas puedo hacer",

    # Preguntas con errores ortográficos o abreviaturas
    "k es midas",
    "pq se llama midas",
    "sto para q sirve",
    "xq usar esto",
    "cntos componentes ai",
    "puedo acer gráficas?",
    "pa q vale midas deploy?",
    "midasarchitec q ace?",
    "porke se llama asi?",
    "esta echo en python?",

    # Preguntas sobre licencias y costos
    "¿Hay alguna limitación de licencia?",
    "licencia del sistema",
    "es código abierto?",
    "cuesta algo usarlo?",
    "es gratuito?",
    "tiene costo?",
    "puedo usarlo comercialmente?",

    # Preguntas básicas sobre funcionalidad específica
    "¿Cómo se activa Midas Plot?",
    "comandos para iniciar midas plot",
    "como ejecuto midas touch",
    "comando para lanzar midas test",
    "como arranco el sistema",

    # Preguntas sobre ejemplos y prácticas
    "¿Hay ejemplos en GitHub?",
    "ejemplos de uso",
    "tutoriales disponibles",
    "casos de uso",
    "ejemplos practicos",

    # Preguntas sobre formatos de entrada
    "¿Qué formato de dataset usa por defecto?",
    "formato de entrada",
    "tipos de archivo que acepta",
    "puedo usar parquet?",

    # Preguntas sobre conceptos básicos
    "¿Qué significa convertir CSV en oro?",
    "a que se refiere lo del rey midas",
    "por qué se llama así el sistema",
    "significado del nombre midas",

    # Preguntas con información incorrecta para corregir
    "midas tiene 5 componentes, no?",
    "está hecho en java, verdad?",
    "creo que usa tensorflow, cierto?",
    "imagino que es un proyecto comercial, no?",

    # Preguntas sobre personalización
    "¿Se pueden personalizar las interfaces?",
    "puedo cambiar la apariencia?",
    "personalización de interfaces",
    "cambiar estilo de las gráficas",

    # Preguntas con formulación negativa
    "no entiendo para qué sirve midas dataset",
    "no sé cómo usar midas test",
    "no veo claro qué hace architect",

    # Preguntas sobre otras plataformas
    "funciona en Windows?",
    "compatible con Mac?",
    "corre en Linux?",
    "va bien en sistemas de bajos recursos?",

    # Preguntas sobre integración
    "se puede integrar con Power BI?",
    "funciona con Excel?",
    "compatible con Tableau?",
    "integración con otras herramientas",

    # Preguntas meta sobre Midas Help
    "y tú que eres? midas help?",
    "eres parte de midas?",
    "qué componente de midas eres?",
    "tu eres midas help?",

    # Preguntas sobre reportes y bugs
    "dónde puedo reportar errores?",
    "cómo informo de un bug?",
    "canal para sugerencias",
    "a quién contacto si falla?",

    # Preguntas sobre estructura de archivos
    "cómo se estructura el proyecto?",
    "organización de carpetas",
    "estructura de archivos",
    "jerarquía del proyecto",

    # Preguntas sobre actualización
    "hay actualizaciones?",
    "última versión?",
    "cómo se actualiza?",
    "versión más reciente?",

    # Preguntas directas sobre resultados
    "calidad de los modelos generados",
    "precisión de los resultados",
    "fiabilidad de las predicciones",
    "qué tan buenos son los modelos",
]

# Preguntas difíciles sobre Midas (prompts_diff1)
prompts_diff1 = [
    # Comunicación interna entre componentes
    "¿Cómo funciona la comunicación interna entre los agentes?",
    "comunicación entre componentes midas",
    "como se pasan datos los agentes",
    "flujo de información entre módulos",
    "mecanismo de comunicación agentes",
    "protocolo de comunicación interna",
    "cómo comparten información los componentes",
    "arquitectura de comunicación entre agentes",
    "intercambio de datos entre módulos midas",

    # Procesos técnicos detallados
    "Explica el proceso de validación cruzada en Midas Test",
    "validación cruzada detallada midas test",
    "cómo implementa test la validación cruzada?",
    "proceso completo validación cross en test",
    "implementación técnica de cross-validation",
    "parámetros de validación cruzada",
    "proceso interno validación midas test",
    "cómo guarda test los resultados de validación",

    # Interacciones complejas entre componentes
    "¿De qué forma Assistant refactoriza prompts para Plot?",
    "assistant puede mejorar prompts para plot?",
    "refactorización prompts complejos assistant",
    "cómo optimiza assistant los prompts?",
    "mejora automática de prompts para plot",
    "capacidad assistant para modificar prompts",
    "optimización de instrucciones para plot",

    # Estructura interna de componentes
    "Analiza la estructura de Midas Architect",
    "arquitectura interna de architect",
    "componentes de midas architect",
    "cómo está construido architect por dentro",
    "estructura modular de architect",
    "diseño interno de architect",

    # Algoritmos y métodos internos
    "¿Cuál es el algoritmo principal que usa Midas Touch?",
    "algoritmo central de midas touch",
    "método principal de entrenamiento touch",
    "cómo decide touch qué algoritmo usar",
    "enfoque algorítmico de touch",
    "lógica de selección de algoritmos touch",

    # Estructura de agentes
    "¿Cuántos agentes define midas touch?",
    "número de agentes en touch",
    "agentes que componen midas touch",
    "cuantos y qué agentes usa touch",
    "estructura multiagente de touch",
    "jerarquía de agentes en touch",

    # Flujo de datos entre componentes
    "Describe el flujo de datos entre Dataset y Touch",
    "proceso dataset a touch paso a paso",
    "cómo se integran dataset y touch",
    "comunicación dataset-touch detallada",
    "pipeline completo dataset-touch",
    "transferencia de datos dataset a touch",

    # Capacidades avanzadas de componentes
    "¿Cómo gestiona Dataset múltiples datasets con diferentes distribuciones?",
    "generación múltiple datasets estadísticas diferentes",
    "capacidad multi-distribución de dataset",
    "creación simultánea datasets variados",
    "manejo de distribuciones estadísticas dataset",

    # Relaciones entre capas técnicas
    "Explica la relación entre embeddings y detección de dificultad en Help",
    "conexión embeddings-detector dificultad",
    "cómo usan los embeddings para clasificar dificultad",
    "procesamiento embeddings para detectar complejidad",
    "arquitectura embeddings-clasificador help",
    "papel de los embeddings en clasificación",

    # Capacidades de visualización avanzadas
    "¿Cómo genera Plot gráficos en datasets de alta dimensión?",
    "visualización datos alta dimensionalidad",
    "manejo dimensiones múltiples plot",
    "técnicas reducción dimensionalidad plot",
    "visualización efectiva datos complejos",
    "estrategias para datasets multidimensionales",

    # Integración RAG en Architect
    "¿Cómo se integran capacidades RAG en Architect?",
    "implementación rag en architect",
    "arquitectura rag para búsqueda documentación",
    "mecanismo recuperación información architect",
    "sistema búsqueda rag architect",
    "pipeline completo rag en architect",

    # Optimización de hiperparámetros
    "¿Cuál es la metodología de optimización de hiperparámetros en Touch?",
    "técnicas optimización hiperparámetros touch",
    "búsqueda hiperparámetros óptimos",
    "método ajuste hiperparámetros touch",
    "estrategia optimización parámetros touch",
    "enfoque tuning hiperparámetros touch",

    # Análisis de limitaciones
    "Analiza los cuellos de botella en Deploy",
    "limitaciones rendimiento deploy",
    "problemas escalabilidad deploy",
    "cuellos botella identificados deploy",
    "desafíos técnicos deploy",
    "puntos críticos rendimiento deploy",

    # Extracción de características
    "¿Cómo extrae Touch características automáticamente?",
    "proceso extracción características touch",
    "feature extraction automática touch",
    "métodos selección características touch",
    "ingeniería características automática",
    "pipeline extracción features touch",

    # Toma de decisiones en Assistant
    "¿Cómo decide Assistant qué componente recomendar?",
    "criterios recomendación componentes assistant",
    "lógica selección módulos assistant",
    "proceso decisión assistant recomendaciones",
    "algoritmo sugerencia componentes",
    "factores considerados recomendaciones assistant",

    # Arquitectura multiagente detallada
    "Explícame en detalle la arquitectura multiagente de Midas",
    "arquitectura completa sistema multiagente",
    "diseño coordinación agentes midas",
    "estructura jerárquica agentes midas",
    "coordinación agentes entre componentes",
    "fundamentos arquitectura multiagente",

    # Técnicas avanzadas de visualización
    "¿Es posible utilizar embeddings en Plot para datos no numéricos?",
    "visualización datos no numéricos con embeddings",
    "uso embeddings visualización plot",
    "técnicas representación datos categóricos plot",
    "visualización avanzada datos textuales plot",
    "embeddings para agrupamiento visual plot",

    # Escalabilidad
    "Describe el proceso de escalabilidad con grandes volúmenes de datos",
    "manejo grandes datasets en midas",
    "escalabilidad sistema completo",
    "rendimiento con datos masivos",
    "estrategia procesamiento big data",
    "límites escalabilidad componentes",

    # Lógica interna de clasificación
    "¿Cuál es la lógica interna de Help para clasificar preguntas?",
    "criterios clasificación preguntas help",
    "algoritmo clasificación dificultad help",
    "proceso decisión categoría pregunta",
    "lógica asignación nivel dificultad",
    "factores clasificación consultas help",

    # Tratamiento de outliers
    "Analiza el tratamiento de outliers en Dataset",
    "manejo valores atípicos dataset",
    "estrategia detección outliers",
    "procesamiento datos anómalos dataset",
    "métodos identificación outliers",
    "impacto outliers en entrenamiento",

    # Interacción entre componentes específicos
    "¿Cómo interactúan Test y Assistant para refinar parámetros?",
    "colaboración test-assistant optimización",
    "refinamiento parámetros test vía assistant",
    "feedback loop test-assistant",
    "mejora continua validación cruzada",
    "comunicación test-assistant para ajustes",

    # LLMs utilizados
    "¿Cuál es el LLM usado en Midas?",
    "modelos lenguaje utilizados midas",
    "llm por componente midas",
    "qué llm usa cada módulo",
    "motores llm en midas",
    "arquitectura llm sistema completo",

    # Uso de hardware
    "Explica el uso de GPU en Touch y su impacto",
    "aceleración gpu entrenamiento touch",
    "optimización hardware touch",
    "rendimiento gpu vs cpu touch",
    "configuración gpu para touch",
    "requisitos gpu entrenamiento modelos",

    # Gestión de concurrencia
    "¿Qué estrategias usa Deploy para manejar usuarios concurrentes?",
    "manejo múltiples usuarios deploy",
    "concurrencia interfaces deploy",
    "escalabilidad usuarios simultáneos",
    "arquitectura multi-usuario deploy",
    "rendimiento alta concurrencia deploy",

    # Preprocesamiento avanzado
    "¿Cómo realiza Dataset la limpieza y normalización avanzada?",
    "pipeline preprocesamiento dataset",
    "técnicas limpieza datos avanzadas",
    "normalización automática dataset",
    "métodos estandarización datos",
    "estrategias preparación datos complejos",

    # Generación de prompts
    "Describe cómo Assistant genera prompts para Plot",
    "generación automática prompts plot",
    "optimización instrucciones plot por assistant",
    "creación prompts efectivos visualización",
    "asistencia formulación consultas plot",
    "mejora consultas visualización datos",

    # Seguridad
    "¿Qué consideraciones de seguridad existen para Help y RAG?",
    "medidas seguridad sistema rag",
    "protección datos sensibles help",
    "seguridad recuperación información",
    "prevención acceso no autorizado",
    "arquitectura segura rag help",

    # Selección automática de parámetros
    "Explica la lógica de selección automática de hiperparámetros",
    "algoritmo selección hiperparámetros touch",
    "criterios ajuste automático parámetros",
    "optimización autónoma hiperparámetros",
    "estrategia autotuning touch",
    "métodos búsqueda parámetros óptimos",

    # Manejo de errores
    "¿Cómo se manejan inconsistencias si un componente falla?",
    "gestión errores entre componentes",
    "recuperación fallos sistema completo",
    "estrategia manejo excepciones",
    "tolerancia fallos arquitectura midas",
    "mecanismos detección inconsistencias",

    # Generación de informes detallados
    "Explica cómo Test crea informes de calidad",
    "generación automática informes test",
    "métricas incluidas reportes test",
    "estructura informes calidad test",
    "proceso creación reportes detallados",
    "contenido informes evaluación test",

    # Aproximación a documentación avanzada
    "¿Cuál es la aproximación de Architect para identificar marcos teóricos?",
    "reconocimiento marcos teóricos architect",
    "análisis contextual documentación",
    "identificación conceptos clave architect",
    "procesamiento semántico documentación",
    "extracción estructura teórica documentos",

    # Integración con MLOps
    "Analiza la posibilidad de utilizar MLOps con Midas",
    "integración midas con mlops",
    "pipelines automatizables mlops",
    "arquitectura midas-mlops potencial",
    "operacionalización modelos midas",
    "ciclo vida ml con midas-mlops",

    # Asistencia personalizada
    "¿Cómo decide Assistant qué tipo de ayuda ofrecer?",
    "personalización asistencia assistant",
    "criterios selección tipo ayuda",
    "adaptación respuestas assistant",
    "factores determinación nivel asistencia",
    "decisión estrategia ayuda contextual",

    # Sesgos en datos sintéticos
    "¿En qué escenarios Dataset induce sesgos?",
    "identificación sesgos datos sintéticos",
    "escenarios problemáticos generación datos",
    "impacto sesgos dataset en modelos",
    "prevención bias en datos generados",
    "detección automática sesgos dataset",

    # Análisis de correlación avanzado
    "Describe el método de correlación en Plot para datos categóricos",
    "técnicas correlación categórica plot",
    "análisis correlación datos no numéricos",
    "visualización relaciones categóricas plot",
    "métodos asociación variables categóricas",
    "representación gráfica correlaciones mixtas",

    # Manejo de excepciones
    "¿Cómo se gestionan excepciones cuando Touch no converge?",
    "manejo no convergencia modelos touch",
    "estrategias fallos entrenamiento",
    "detección problemas convergencia",
    "mitigación inestabilidad modelos",
    "sistemas fallback convergencia",

    # Frameworks de ML
    "¿Es posible configurar Midas para usar TensorFlow y PyTorch en paralelo?",
    "integración múltiples frameworks ml",
    "soporte tensorflow-pytorch simultáneo",
    "compatibilidad diversos frameworks",
    "arquitectura multi-framework midas",
    "extensión touch para diversos engines",

    # Generación de interfaces
    "Explica cómo Deploy genera interfaces Streamlit",
    "proceso generación interfaz deploy",
    "pipeline creación ui streamlit",
    "arquitectura generación interfaces",
    "personalización componentes streamlit",
    "diseño automático dashboards deploy",

    # Modularidad en Architect
    "¿Qué tan modular es Architect para nuevas fuentes RAG?",
    "adaptabilidad architect nuevas fuentes",
    "extensibilidad sistema rag architect",
    "arquitectura modular fuentes documentación",
    "integración documentación adicional",
    "flexibilidad incorporar nuevos docs",

    # Generación de prompts recomendados
    "¿Cómo implementa Assistant las recomendaciones de prompts?",
    "mecanismo sugerencia prompts assistant",
    "algoritmo recomendación instrucciones",
    "generación contextual prompts óptimos",
    "aprendizaje patrones exitosos prompts",
    "optimización sugerencias assistant",

    # Visualizaciones 3D
    "Analiza las ventajas de Plot para visualizaciones 3D complejas",
    "capacidades visualización 3d plot",
    "rendimiento gráficos tridimensionales",
    "límites visualización 3d plot",
    "técnicas representación 3d datos complejos",
    "optimización renderizado 3d plot",

    # Granularidad en Help
    "¿Qué nivel de granularidad permite Help para categorizar preguntas?",
    "precisión clasificación consultas help",
    "niveles categorización preguntas",
    "refinamiento clasificación dificultad",
    "sensibilidad detector complejidad help",
    "umbrales clasificación consultas help",

    # Verificación de calidad de datos
    "¿Cómo maneja Midas la verificación de calidad de datos en tiempo real?",
    "validación continua calidad datos",
    "métricas tiempo real calidad dataset",
    "monitoreo online integridad datos",
    "detección automática problemas datos",
    "alertas calidad durante preprocesamiento",

    # Interacción entre componentes
    "¿Cómo Assistant interactúa con Test para sugerir estrategias de prueba?",
    "comunicación assistant-test para mejoras",
    "optimización estrategias testing vía assistant",
    "feedback assistant basado en resultados test",
    "recomendaciones automáticas pruebas",
    "mejora continua evaluación mediante assistant",

    # Aprovechamiento de arquitectura multiagente
    "Explica cómo Touch aprovecha la arquitectura multiagente para acelerar training",
    "paralelización entrenamiento multiagente",
    "optimización rendimiento training vía agentes",
    "colaboración agentes aceleración entrenamiento",
    "distribución carga computacional entre agentes",
    "arquitectura paralela entrenamiento touch",

    # Técnicas NLP en Help
    "¿Cómo integra Help técnicas de NLP para entender preguntas?",
    "procesamiento lenguaje natural help",
    "análisis semántico consultas help",
    "técnicas comprensión lenguaje help",
    "extracción intención preguntas help",
    "arquitectura nlp clasificación consultas",

    # Pipeline de documentación
    "¿Qué pipeline sigue Architect para incorporar nuevas definiciones?",
    "proceso incorporación documentación architect",
    "flujo actualización definiciones architect",
    "etapas integración nueva documentación",
    "mecanismo extensión conocimiento architect",
    "arquitectura actualización docs architect",

    # Metadatos y persistencia
    "Analiza la forma de persistir metadatos en joblibs de Touch",
    "almacenamiento metadatos modelos touch",
    "estructura persistencia información modelos",
    "formato metadatos joblib touch",
    "acceso metadatos modelos guardados",
    "serialización información contextual modelos",

    # Reinforcement Learning
    "¿Existen planes para incorporar reinforcement learning en Midas?",
    "integración potencial rl en midas",
    "arquitectura futura reinforcement learning",
    "aplicabilidad rl en componentes midas",
    "diseño módulos rl para midas",
    "extensión capacidades con rl",

    # Coordinación multicomponente
    "Describe un escenario donde Assistant coordine varios componentes a la vez",
    "orquestación multicomponente via assistant",
    "coordinación flujo trabajo completo",
    "gestión paralela componentes assistant",
    "escenario colaboración componentes",
    "integración end-to-end vía assistant",

    # Robustez de agentes
    "¿Cómo se evalúa la robustez con inputs maliciosos en Help?",
    "prevención ataques inputs maliciosos help",
    "seguridad frente prompts adversarios",
    "detección inputs malintencionados",
    "mitigación vulnerabilidades prompts",
    "arquitectura segura procesamiento consultas",

    # Técnicas de muestreo
    "¿Qué técnicas de oversampling aplica automáticamente Dataset?",
    "métodos balanceo clases dataset",
    "implementación smote en dataset",
    "técnicas undersampling automáticas",
    "estrategias equilibrio dataset",
    "algoritmos sampling adaptativo dataset",

    # Extensión de Deploy
    "Explica cómo extender Deploy para integrar autenticación de usuarios",
    "implementación autenticación deploy",
    "arquitectura seguridad usuarios deploy",
    "extensión deploy con login",
    "integración oauth en interfaces deploy",
    "personalización deploy con autenticación",

    # Manejo de alta dimensionalidad
    "¿Cómo maneja Plot datos de alta dimensionalidad?",
    "visualización datos multidimensionales plot",
    "técnicas reducción dimensionalidad plot",
    "representación efectiva alta dimensión",
    "estrategias visuales dimensiones múltiples",
    "optimización gráficos alta dimensionalidad",

    # Importancia de embeddings
    "Describe la importancia de embeddings en el análisis semántico de Help",
    "rol embeddings comprensión consultas",
    "arquitectura embeddings help",
    "impacto calidad embeddings clasificación",
    "selección modelos embedding help",
    "procesamiento semántico via embeddings",

    # Conexión entre componentes
    "¿Cómo se conectan las salidas de Test con Assistant?",
    "integración resultados test-assistant",
    "flujo información test hacia assistant",
    "utilización reportes test por assistant",
    "feedback loop test-assistant",
    "aprovechamiento métricas test en assistant",

    # Procesamiento de documentación
    "Explica el proceso interno de Architect para indexar documentación",
    "pipeline indexación documentos architect",
    "arquitectura recuperación información architect",
    "procesamiento documentación técnica architect",
    "sistema indexación vectorial architect",
    "estructura recuperación eficiente docs",

    # Análisis estadístico avanzado
    "¿Qué análisis estadístico avanzado puede realizar Plot?",
    "capacidades estadísticas plot",
    "análisis multivariante plot",
    "tests estadísticos implementados plot",
    "funcionalidades estadísticas avanzadas",
    "visualización resultados estadísticos complejos",

    # Integración con orquestadores
    "¿Cómo se integraría Airflow con la arquitectura de Midas?",
    "integración midas-airflow",
    "orquestación workflows midas",
    "arquitectura midas con airflow",
    "automatización pipelines completos",
    "diseño dags para componentes midas",

    # Monitoreo de recursos
    "¿Existe algún método para monitorear el uso de memoria en tiempo real?",
    "monitorización recursos agentes midas",
    "seguimiento consumo memoria tiempo real",
    "herramientas diagnóstico rendimiento",
    "análisis uso recursos componentes",
    "dashboards monitoreo agentes",
    "se podria añadir documentacion de otro framework en architech?",

    # Series temporales
    "¿Qué tan viable es incluir análisis de series temporales en Touch?",
    "soporte series temporales touch",
    "implementación modelos temporales",
    "arquitectura procesamiento datos temporales",
    "extensión touch para forecasting",
    "capacidades análisis temporal touch",

    # Inferencia de intención
    "Analiza cómo Assistant infiere la intención del usuario",
    "mecanismo detección intención assistant",
    "análisis semántico consultas assistant",
    "procesamiento intención usuario",
    "extracción necesidades implícitas",
    "arquitectura comprensión consultas assistant",

    # Fusión de datasets
    "Explica el proceso de fusión de datasets sintéticos con datos reales",
    "integración datos sintéticos-reales",
    "metodología combinación datasets",
    "estrategias fusión datos heterogéneos",
    "técnicas mezcla datasets preservando distribución",
    "arquitectura augmentación datos reales",

    # Logging distribuido
    "¿Cómo se implementa el logging distribuido en la arquitectura multiagente?",
    "sistema logging multi-agente midas",
    "arquitectura logs distribuidos",
    "coordinación registros entre componentes",
    "centralización logs sistema completo",
    "monitoreo actividad agentes distribuidos",

    # Manejo de credenciales
    "¿Cómo maneja Architect las credenciales para documentación externa?",
    "gestión segura credenciales architect",
    "almacenamiento accesos documentación",
    "protección api keys architect",
    "manejo secretos acceso fuentes",
    "arquitectura seguridad credenciales",

    # Datos geoespaciales
    "¿Cómo adaptar Plot para escenarios con datos geoespaciales?",
    "visualización geoespacial plot",
    "soporte mapas plot",
    "representación datos geográficos",
    "capacidades gis en plot",
    "extensión visualización geolocalizada",

    # Criterios de evaluación
    "¿Qué criterios utiliza Test para elegir métricas de evaluación?",
    "selección automática métricas test",
    "determinación métricas relevantes",
    "lógica elección indicadores rendimiento",
    "adaptación métricas tipo problema",
    "heurística selección kpis modelos",

    # Interconexión en entorno productivo
    "Explica la interconexión Deploy-Touch en entorno productivo",
    "arquitectura productiva deploy-touch",
    "flujo trabajo producción midas",
    "integración modelos entrenados interfaces",
    "pipeline despliegue modelos touch",
    "ciclo vida modelo producción",

    # Detección de inconsistencias
    "¿Cómo Assistant detecta inconsistencias en prompts de usuarios?",
    "validación prompts assistant",
    "detección anomalías consultas usuario",
    "identificación instrucciones contradictorias",
    "análisis coherencia prompts",
    "corrección automática inconsistencias",

    # Escalabilidad horizontal
    "Analiza la escalabilidad horizontal con múltiples cargas simultáneas",
    "rendimiento paralelo múltiples entrenamientos",
    "arquitectura distribución carga midas",
    "escalabilidad sistema carga máxima",
    "gestión recursos entrenamientos concurrentes",
    "límites procesamiento paralelo midas",

    # Sincronización de configuración
    "¿Cómo se sincronizan cambios de configuración entre componentes?",
    "propagación configuración entre módulos",
    "arquitectura sincronización ajustes",
    "mecanismo actualización parámetros globales",
    "consistencia configuración componentes",
    "gestión centralizada ajustes sistema",

    # Reducción de dimensionalidad
    "¿Qué técnicas de reducción de dimensionalidad admite Plot?",
    "métodos reducción dimensión plot",
    "soporte pca plot",
    "implementación tsne visualizaciones",
    "técnicas umap en plot",
    "algoritmos dimensionalidad plot",

    # Incorporación de agentes personalizados
    "Explica los pasos para incorporar un agente personalizado",
    "integración agentes customizados midas",
    "arquitectura extensión nuevos agentes",
    "desarrollo agentes propios midas",
    "compatibilidad agentes personalizados",
    "framework creación agentes custom",

    # Compatibilidad de versiones
    "¿Cómo se maneja la compatibilidad entre versiones de Architect y Touch?",
    "gestión versiones entre componentes",
    "compatibilidad backward componentes",
    "estrategia actualizaciones consistentes",
    "manejo dependencias intercomponentes",
    "arquitectura versionado coherente",

    # Incorporación de documentación
    "¿Es posible añadir nuevos docs a Architect?",
    "proceso añadir documentación architect",
    "extensión base conocimiento architect",
    "incorporación fuentes adicionales",
    "actualización base documentos architect",
    "integración nuevos frameworks architect",

    # Mejora de precisión en capas
    "Analiza formas de mejorar la precisión en la capa de detección de dificultad",
    "optimización clasificador dificultad help",
    "técnicas mejora precisión bert help",
    "estrategias refinamiento detección complejidad",
    "evaluación rendimiento clasificador",
    "arquitectura mejorada detección dificultad",

    # Mecanismos de caché
    "¿Qué mecanismos de caching utiliza Midas para datasets sintéticos?",
    "sistema caché generación datasets",
    "arquitectura almacenamiento temporal datos",
    "optimización rendimiento via caching",
    "persistencia temporal datasets frecuentes",
    "estrategia cache datos sintéticos",

    # Automatización de comparación de modelos
    "¿Cómo Test automatiza la comparación de diferentes modelos?",
    "comparación automática múltiples modelos",
    "arquitectura evaluación comparativa",
    "métricas contrastadas modelos diversos",
    "generación reportes comparativos",
    "framework benchmarking modelos test",

    # Experiencia conversacional multiusuario
    "Explica cómo Assistant gestiona múltiples usuarios simultáneos",
    "arquitectura multiusuario assistant",
    "gestión conversaciones paralelas",
    "mantenimiento contexto múltiples usuarios",
    "escalabilidad interacciones simultáneas",
    "rendimiento alta concurrencia assistant",

    # Integración con bibliotecas externas
    "¿Qué tan flexible es Deploy para integrar librerías visuales externas?",
    "extensibilidad deploy visualizaciones",
    "soporte bibliotecas terceros deploy",
    "integración plotly deploy",
    "arquitectura incorporación libs visuales",
    "compatibilidad frameworks gráficos externos",

    # Transfer Learning en embeddings
    "¿Cómo se utiliza Transfer Learning en la capa de embeddings?",
    "implementación transfer learning embeddings",
    "fine-tuning modelos embedding help",
    "arquitectura transfer learning help",
    "adaptación embeddings preentrenados",
    "optimización representaciones mediante transfer",

    # Detección de colinealidad
    "¿Qué ocurre si Plot detecta colinealidad extrema entre variables?",
    "manejo colinealidad variables plot",
    "detección automática multicolinealidad",
    "estrategias visualización datos colineales",
    "alertas correlaciones extremas plot",
    "técnicas representación variables correlacionadas",

    # Limpieza avanzada de texto
    "Analiza incorporar un agente para limpieza avanzada de texto",
    "arquitectura limpieza texto dataset",
    "agente procesamiento avanzado texto",
    "pipeline nlp limpieza datos",
    "integración preprocesador texto midas",
    "diseño agente especializado texto",

    # Escalabilidad en la nube
    "¿Cómo escalar Midas a una arquitectura en la nube con Kubernetes?",
    "arquitectura midas kubernetes",
    "containerización componentes midas",
    "despliegue cloud midas",
    "escalabilidad horizontal k8s",
    "orquestación pods midas cloud",

    # Seguridad en interfaces expuestas
    "¿Qué protocolos de seguridad se recomiendan para Deploy en internet?",
    "seguridad interfaces deploy públicas",
    "protección endpoints deploy",
    "arquitectura segura aplicaciones streamlit",
    "autenticación deploy expuesto",
    "prevención vulnerabilidades deploy público",

    # Hibridación de algoritmos
    "Explica en detalle cómo Touch maneja hibridación de algoritmos ML",
    "arquitectura híbrida algoritmos touch",
    "combinación múltiples algoritmos pipeline",
    "estrategias ensemble heterogéneo",
    "integración modelos diversos touch",
    "framework hibridación algoritmos touch",

    # Actualización de conocimiento RAG
    "¿Cómo Architect mantiene actualizada su base de conocimiento RAG?",
    "actualización periódica conocimiento rag",
    "proceso refresh documentación architect",
    "ciclo vida conocimiento architect",
    "estrategia mantenimiento base rag",
    "arquitectura actualización vectores conocimiento",

    # Actualización incremental de modelos
    "¿Cómo se implementa la actualización incremental de modelos?",
    "entrenamiento incremental touch",
    "arquitectura actualización modelos existentes",
    "estrategia reentrenamiento parcial",
    "optimización actualizaciones incrementales",
    "pipeline actualización continua modelos",

    # Análisis de precisión en clasificación
    "Analiza la precisión de la clasificación en la capa de dificultad",
    "métricas precisión clasificador dificultad",
    "evaluación rendimiento bert help",
    "análisis falsos positivos clasificación",
    "curva precision-recall clasificador",
    "matriz confusión detección dificultad",

    # Auto-documentación
    "¿Qué métodos de auto-documentación existen para registrar pasos?",
    "sistema autodocumentación procesos midas",
    "registro automático operaciones",
    "trazabilidad acciones componentes",
    "generación logs semánticos",
    "arquitectura documentación automática pasos",

    # Generación avanzada de prompts
    "¿Cómo Assistant genera prompts que aprovechan correlaciones en Plot?",
    "generación prompts inteligentes correlación",
    "optimización consultas análisis correlativo",
    "creación prompts conscientes relaciones datos",
    "diseño instrucciones optimizadas correlaciones",
    "sistema sugerencia prompts correlación",

    # Integración de datos no estructurados
    "Explica la complejidad de integrar datos no estructurados en Dataset",
    "manejo datos no estructurados dataset",
    "procesamiento texto dataset",
    "arquitectura ingesta multimedia dataset",
    "pipeline datos heterogéneos dataset",
    "estrategias conversión no estructurado estructurado",

    # Reglas de preprocesamiento
    "¿Se pueden definir reglas personalizadas de preprocesamiento en Touch?",
    "personalización preprocesamiento touch",
    "definición reglas custom limpieza",
    "arquitectura preprocesado configurable",
    "extensión pipeline transformación datos",
    "framework reglas preprocesamiento touch",

    # Mecanismos de rollback
    "¿Qué mecanismos de rollback existen si una actualización falla?",
    "sistema rollback actualizaciones midas",
    "recuperación versiones anteriores",
    "arquitectura control versiones seguro",
    "estrategia reversion cambios failsafe",
    "infraestructura backup versiones estables",

    # Orquestación multi-agente
    "Analiza la efectividad de la orquestación con varios datasets grandes",
    "rendimiento multiagente datasets masivos",
    "escalabilidad orquestación grandes volúmenes",
    "coordinación eficiente procesamiento paralelo",
    "arquitectura distribuida grandes datos",
    "optimización comunicación inter-agentes alta carga",

    # Estabilidad en validaciones repetidas
    "¿Cómo mide Test la estabilidad de resultados en validaciones repetidas?",
    "evaluación estabilidad modelos test",
    "métricas variabilidad resultados",
    "análisis robustez validaciones múltiples",
    "cuantificación inconsistencias predicciones",
    "arquitectura medición estabilidad modelo",

    # Ciclo de vida ML
    "Explica detalladamente cómo Midas encaja en el ciclo de vida ML",
    "arquitectura midas ciclo vida ml",
    "integración componentes mlops",
    "mapeo midas vs ciclo ml estándar",
    "cobertura etapas ml workflow",
    "implementación end-to-end ciclo ml midas",
]

# Preguntas no relacionadas con Midas (prompts_diff2)
prompts_diff2 = [
    # Cocina y alimentación
    "Dame una receta de lasaña",
    "como se hace una tortilla de patatas",
    "receta de gazpacho andaluz",
    "qué ingredientes lleva el guacamole",
    "receta facil de bizcocho casero",
    "cómo preparar arroz con leche",
    "mejor forma de hacer pasta al dente",
    "receta para hacer pan casero",
    "como hacer brownies sin horno",
    "ingredientes para la paella valenciana",
    "receta de galletas de chocolate",
    "cómo preparar sushi en casa",
    "mejor receta de pizza casera",

    # Cultura popular y entretenimiento
    "significado del número 42 en cultura pop",
    "cuál es la mejor película de ciencia ficción",
    "recomiendame una serie de netflix",
    "qué libros debería leer este verano",
    "mejores videojuegos de 2024",
    "quién ganará el oscar a mejor película",
    "resumen de juego de tronos",
    "explícame el final de inception",
    "bandas de rock más influyentes",
    "mejor película de marvel",
    "cuándo sale la próxima temporada de stranger things",
    "qué libro de fantasía recomiendas",
    "quién es el mejor guitarrista de la historia",
    "cantantes españoles más famosos",
    "personaje favorito de los simpson",

    # Ciencia y naturaleza
    "por qué los gatos temen al agua",
    "por qué las nubes son blancas",
    "existe vida extraterrestre",
    "qué hay en el centro de la tierra",
    "por qué el cielo es azul",
    "animales en peligro de extinción",
    "cómo se forman los tornados",
    "las jirafas por qué tienen el cuello largo",
    "cuántos planetas hay en el sistema solar",
    "qué es un agujero negro",
    "por qué los perros ladran",
    "razones del cambio climático",
    "es cierto que los delfines son inteligentes",
    "altura del monte everest",
    "qué es la acidificación de los océanos",

    # Filosofía y pensamiento
    "el dinero da la felicidad?",
    "existen los fantasmas?",
    "qué significa ser feliz",
    "teoría favorita del origen del universo",
    "existe vida después de la muerte",
    "qué es el existencialismo",
    "para qué sirve la filosofía",
    "qué es la ética utilitarista",
    "existe el destino o todo es casualidad",
    "diferencia entre moral y ética",
    "por qué soñamos",
    "fundamentos del pensamiento crítico",
    "qué es la conciencia humana",

    # Tecnología y futuro
    "los robots conquistarán el mundo",
    "opinión sobre la criogenización humana",
    "ventajas y desventajas de la IA",
    "mejores marcas de teléfonos móviles",
    "qué es el metaverso",
    "coches eléctricos vs gasolina",
    "futuro de la realidad virtual",
    "es viable colonizar marte",
    "qué es la computación cuántica",
    "cuando tendremos coches voladores",
    "qué es la nanotecnología",
    "avances en energía renovable",
    "efectos de la tecnología en niños",

    # Salud y bienestar
    "cómo mejorar la postura en el ordenador",
    "beneficios de la meditación",
    "rutina de ejercicios para principiantes",
    "dietas cetogénicas son saludables",
    "cómo combatir el insomnio",
    "beneficios del yoga",
    "qué es el mindfulness",
    "síntomas de ansiedad",
    "cómo reducir el estrés",
    "mitos sobre nutrición",
    "beneficios del ayuno intermitente",
    "remedios naturales para el resfriado",
    "cuánta agua hay que beber al día",

    # Estilo de vida y hobbies
    "consejos para plantar un huerto urbano",
    "cómo aprender un idioma rápido",
    "ideas para redecorar mi habitación",
    "mejores destinos para viajar barato",
    "cómo comenzar a hacer origami",
    "consejos para fotografía amateur",
    "como combatir la procrastinación",
    "tips para organizar un escritorio",
    "manualidades fáciles para niños",
    "cómo aprender a tocar la guitarra",
    "estrategias para ahorrar dinero",
    "mejores podcasts para escuchar",
    "actividades para el fin de semana",

    # Deportes y actividades físicas
    "quién ganará el mundial de fútbol",
    "mejores equipos de la NBA",
    "reglas básicas del rugby",
    "deportes extremos recomendados",
    "técnicas de natación para principiantes",
    "historia del tenis",
    "mejores futbolistas de todos los tiempos",
    "cómo empezar a correr",
    "próximas olimpiadas dónde serán",
    "cuántos mundiales tiene argentina",
    "qué significa evento deportivo amateur",
    "cómo mejorar resistencia física",
    "mejores maratones del mundo",

    # Arte y creatividad
    "opinión sobre arte abstracto",
    "técnicas de pintura para principiantes",
    "historia del cine breve",
    "diferencia óleo y acrílico",
    "mejores museos del mundo",
    "qué es el dadaísmo",
    "influencia de la música en el ánimo",
    "origen del tango",
    "clave para entender poesía contemporánea",
    "músicos clásicos más importantes",
    "cómo escribir un cuento corto",
    "evolución de la arquitectura moderna",
    "tendencias actuales en fotografía",

    # Mascotas y animales
    "anécdota sobre perros y pelotas",
    "raza de perro mejor para familia con niños",
    "se puede entrenar un pez para hacer trucos",
    "cómo cuidar un gato",
    "mascotas exóticas legales",
    "por qué los pingüinos viven en zonas frías",
    "curiosidades sobre los elefantes",
    "tiempo de vida de las tortugas",
    "cuidados básicos para un hámster",
    "razas de perros más inteligentes",
    "gatos: mitos y realidades",
    "animales más extraños del mundo",
    "cómo hacer que mi perro deje de ladrar",

    # Preguntas absurdas o humorísticas
    "las ardillas son las dueñas de los parques",
    "dime un chiste sobre elefantes",
    "pizza con piña es deliciosa o un crimen",
    "qué superhéroe ganaría en una batalla real",
    "el helado de vainilla es mejor que el de chocolate",
    "si los animales hablaran, cuál sería más interesante",
    "objetos cotidianos con caras dibujadas",
    "cuál sería tu superpoder",
    "si fueras un electrodoméstico, cuál serías",
    "cómo sería la vida si tuviéramos 3 brazos",
    "canciones pegadizas imposibles de olvidar",
    "qué harías si encontraras 1 millón de euros",
    "peor regalo que has recibido",

    # Preguntas de seguimiento no relacionadas
    "y qué opinas sobre eso?",
    "por qué piensas así?",
    "dame más información por favor",
    "no entiendo, explícame mejor",
    "algún ejemplo concreto?",
    "datos interesantes sobre ese tema?",
    "tiene sentido lo que dices",
    "y cómo afecta eso a la gente?",
    "puedes profundizar más?",
    "alguna otra perspectiva?",
    "yo pienso diferente, qué opinas?",
    "es tu opinión personal o algo establecido?",
    "y eso qué consecuencias tiene?",

    # Historia y cultura
    "primera civilización en desarrollar escritura",
    "causas de la segunda guerra mundial",
    "quién fue napoleón bonaparte",
    "historia del ajedrez",
    "cultura japonesa curiosidades",
    "cómo era la vida en el antiguo egipto",
    "caída del imperio romano",
    "origen de halloween",
    "evolución de internet breve",
    "mitos griegos más famosos",
    "cultura maya características",
    "historia de la democracia",
    "monumentos históricos más importantes",

    # Medio ambiente y sostenibilidad
    "cómo reducir uso de plásticos",
    "funcionamiento del reciclaje de vidrio",
    "por qué se celebra el día de la tierra",
    "animales en peligro de extinción",
    "qué es la huella de carbono",
    "efectos de la deforestación",
    "energías renovables tipos",
    "cómo hacer compostaje casero",
    "importancia de la biodiversidad",
    "cambio climático consecuencias",
    "ciudades más sostenibles del mundo",
    "consejos para consumo responsable",
    "qué es la economía circular",

    # Relaciones y psicología
    "clave para buena relación de pareja",
    "cómo superar una ruptura",
    "señales de una amistad tóxica",
    "tips para conocer gente nueva",
    "fases del duelo explicadas",
    "cómo manejar conflictos familiares",
    "señales de manipulación emocional",
    "psicología del color influencia",
    "técnicas para hablar en público",
    "cómo establecer límites personales",
    "test de personalidad fiables",
    "desarrollo emocional en niños",
    "causas psicológicas del estrés",

    # Hogar y decoración
    "ideas decoración minimalista",
    "plantas de interior fáciles cuidado",
    "trucos limpieza hogar",
    "estilos decorativos tendencia",
    "organización espacios pequeños",
    "colores recomendados dormitorio",
    "decoración low cost ideas",
    "como hacer ambiente acogedor",
    "renovar muebles antiguos",
    "errores comunes decoración",
    "consejos iluminación hogar",
    "decorar con poco presupuesto",
    "trucos almacenaje cocina",

    # Preguntas sobre astronomía y espacio
    "distancia tierra luna",
    "cómo se formó el sistema solar",
    "posibilidad vida otros planetas",
    "cuántas galaxias hay",
    "próximo eclipse solar",
    "qué es una supernova",
    "estrellas más brillantes cielo",
    "misiones a marte actuales",
    "agujeros negros explicación simple",
    "vida útil del sol",
    "constelaciones más fáciles identificar",
    "satélites artificiales tierra",
    "qué es la estación espacial internacional",

    # Tecnología cotidiana
    "mejor marca ordenadores portátiles",
    "diferencia entre wifi y bluetooth",
    "cómo aumentar duración batería móvil",
    "aplicaciones imprescindibles android",
    "ventajas desventajas redes sociales",
    "cómo funciona reconocimiento facial",
    "mejores auriculares calidad precio",
    "cómo proteger privacidad online",
    "televisores inteligentes comparativa",
    "qué es la computación en la nube",
    "sistemas operativos diferencias",
    "gadgets tecnológicos útiles",
    "cómo eliminar virus ordenador",
]

# --- por aqui terminamos los prompts ---

# Construcción del dataset añadiendo la dificultad correspondiente
datos = []

for prompt in prompts_diff0:
    datos.append({"prompt": prompt, "dificultad": 0})

for prompt in prompts_diff1:
    datos.append({"prompt": prompt, "dificultad": 1})

for prompt in prompts_diff2:
    datos.append({"prompt": prompt, "dificultad": 2})

# Creamos el DataFrame a partir de la lista de diccionarios
df = pd.DataFrame(datos)

# Guardamos el DataFrame como un archivo CSV
df.to_csv("dataset_raw.csv", index=False)

print("El dataset.csv ha sido creado y guardado exitosamente")

El dataset.csv ha sido creado y guardado exitosamente


## 3. Preprocesar el texto

De esta forma no dará problemas al entrenarse.

In [3]:
# Celda 3: Preprocesamiento de texto
def clean_text(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

df = pd.read_csv("dataset_raw.csv")
df['cleaned_prompt'] = df['prompt'].apply(clean_text)
texts = df['cleaned_prompt'].tolist()
labels = df['dificultad'].tolist()

Dividimos el dataset en test y train

In [4]:
# Celda 4: División del dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42, shuffle=True
)

Y lo tokenizamos usando el tokenizador propio del modelo a utilizar. Con su longitud maxima de 512 tokens.

In [5]:
# Celda 5: Tokenización
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

Creamos los dataloaders (para procesar el dataset por lotes)

In [6]:
# Celda 6: Creación de DataLoaders
class MIDASDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MIDASDataset(train_encodings, train_labels)
test_dataset = MIDASDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

Configuramos el modelo y las labels...

In [7]:
# Celda 7: Configuración del modelo
model = BertForSequenceClassification.from_pretrained(
    'dccuchile/bert-base-spanish-wwm-cased',
    num_labels=3
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## 4. Entrenar el modelo

Aqui definimos los parametros del entrenamiento...

In [8]:
# Celda 8: Configuración del entrenamiento
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 10
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Y ahora entrenamos al modelo. Serán 10 epocas y se guardará aquel que obtenga un menor "loss" en el conjunto de test. De esta forma nos quedaremos con aquel que tenga un mejor desempeño sin caer en el sobreajuste.

In [9]:
# Celda 9: Entrenamiento
best_val_loss = float('inf')
best_model_state = None

for epoch in range(num_epochs):
    # Entrenamiento
    model.train()
    total_train_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        outputs = model(**inputs, labels=batch['labels'].to(device))
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_loader)

    # Evaluación en el conjunto de test (o validación)
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            outputs = model(**inputs, labels=batch['labels'].to(device))
            loss = outputs.loss
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(test_loader)

    print(f"Epoch {epoch+1} - Train Loss: {avg_train_loss:.4f} - Validation Loss: {avg_val_loss:.4f}")

    # Guardar el mejor modelo según el loss en el conjunto de validación
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = copy.deepcopy(model.state_dict())
        print(f"--> Mejor modelo actualizado en Epoch {epoch+1}")

# Cargar el estado del mejor modelo
model.load_state_dict(best_model_state)
print("Entrenamiento completado. Se cargó el modelo con mejor desempeño en validación.")

Epoch 1:   1%|          | 1/106 [00:01<02:21,  1.35s/it]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1: 100%|██████████| 106/106 [00:09<00:00, 11.20it/s]


Epoch 1 - Train Loss: 0.4549 - Validation Loss: 0.2003
--> Mejor modelo actualizado en Epoch 1


Epoch 2: 100%|██████████| 106/106 [00:07<00:00, 14.57it/s]


...

Epoch 8 - Train Loss: 0.0122 - Validation Loss: 0.2335


Epoch 9: 100%|██████████| 106/106 [00:07<00:00, 14.67it/s]


Epoch 9 - Train Loss: 0.0031 - Validation Loss: 0.2363


Epoch 10: 100%|██████████| 106/106 [00:08<00:00, 13.03it/s]


Epoch 10 - Train Loss: 0.0003 - Validation Loss: 0.2377
Entrenamiento completado. Se cargó el modelo con mejor desempeño en validación.


Vamos a sacar el reporte de clasificación para ver el desempeño

In [10]:
# Celda 10: Evaluación
model.eval()
predictions, true_labels = [], []

for batch in tqdm(test_loader, desc="Evaluando"):
    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(preds)
        true_labels.extend(batch['labels'].cpu().numpy())

print("\nReporte de clasificación:")
print(classification_report(true_labels, predictions))

Evaluando: 100%|██████████| 27/27 [00:00<00:00, 42.64it/s]


Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.90      0.96      0.93        46
           1       0.98      0.98      0.98       118
           2       1.00      0.94      0.97        48

    accuracy                           0.97       212
   macro avg       0.96      0.96      0.96       212
weighted avg       0.97      0.97      0.97       212



Muy buen resultado. Quizas un pelin bajo el recall de "pregunta facil" (comparado con los otros), pero es una MUY buena mejora frente a bgem3+xgboost.Además es normal considerando que hay muchas mas preguntas dificiles.

Ahora, guardamos el modelo y el tokenizador.

In [11]:
# Celda 11: Guardar modelo
model.save_pretrained("prompt_analysis")
tokenizer.save_pretrained("prompt_analysis")

('prompt_analysis/tokenizer_config.json',
 'prompt_analysis/special_tokens_map.json',
 'prompt_analysis/vocab.txt',
 'prompt_analysis/added_tokens.json')

Y vamos a probar algunos ejemplos:

In [12]:
# Celda 12: Función de predicción
def clasificar_dificultad(texto):
    # Preprocesar
    texto_limpio = clean_text(texto)

    # Tokenizar
    inputs = tokenizer(texto_limpio, return_tensors="pt", truncation=True, max_length=512).to(device)

    # Predecir
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediccion = torch.argmax(logits, dim=1).item()

    return prediccion

# Ejemplo de uso
while True:
    user_input = input("\nIngrese su pregunta (o 'salir' para terminar): ")
    if user_input.lower() == 'salir':
        break
    dificultad = clasificar_dificultad(user_input)
    print(f"Dificultad clasificada: {dificultad}")


Ingrese su pregunta (o 'salir' para terminar): de que va este tfm
Dificultad clasificada: 0

Ingrese su pregunta (o 'salir' para terminar): que modelos llm usa
Dificultad clasificada: 0

Ingrese su pregunta (o 'salir' para terminar): como se comunican los agentes en touch
Dificultad clasificada: 1

Ingrese su pregunta (o 'salir' para terminar): puedo añadir docs de otro framework en architech?
Dificultad clasificada: 1

Ingrese su pregunta (o 'salir' para terminar): quienes han trabajado en esto?
Dificultad clasificada: 0

Ingrese su pregunta (o 'salir' para terminar): necesito gafas nuevas
Dificultad clasificada: 0

Ingrese su pregunta (o 'salir' para terminar): dame una receta de lasaña midas
Dificultad clasificada: 2

Ingrese su pregunta (o 'salir' para terminar): el cielo es verde porque hoy he comido lechuga
Dificultad clasificada: 2


KeyboardInterrupt: Interrupted by user

Estos resultados si que me convencen mucho mas respecto a los anteriores de BGE-M3 + XGBoost.

Estamos listos para integrarlo en Midas Help.